 ### 적은 양의 데이터로 training 까지 오류없이 진행되는지 확인.

In [7]:
from datasets import load_dataset

billsum = load_dataset("Kyudan/GTNT_8.25M")
billsum["train"] = billsum["train"].select(range(1000))

split_dataset = billsum['train'].train_test_split(test_size=0.2)

# train과 test 데이터셋 확인
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

def preprocess_function(examples):
    # Prepends the string "summarize: " to each document in the 'text' field of the input examples.
    # This is done to instruct the T5 model on the task it needs to perform, which in this case is summarization.
    inputs = ["translate: " + doc for doc in examples["NT"]]

    # Tokenizes the prepended input texts to convert them into a format that can be fed into the T5 model.
    # Sets a maximum token length of 1024, and truncates any text longer than this limit.
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)

    # Tokenizes the 'summary' field of the input examples to prepare the target labels for the summarization task.
    # Sets a maximum token length of 128, and truncates any text longer than this limit.
    labels = tokenizer(text_target=examples["GT"], max_length=256, truncation=True)

    # Assigns the tokenized labels to the 'labels' field of model_inputs.
    # The 'labels' field is used during training to calculate the loss and guide model learning.
    model_inputs["labels"] = labels["input_ids"]

    # Returns the prepared inputs and labels as a single dictionary, ready for training.
    return model_inputs

tokenized_billsum = split_dataset.map(preprocess_function, batched=True)
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="facebook/bart-base")
import evaluate

rouge = evaluate.load("rouge")
import numpy as np

def compute_metrics(eval_pred):
    # Unpacks the evaluation predictions tuple into predictions and labels.
    predictions, labels = eval_pred

    # Decodes the tokenized predictions back to text, skipping any special tokens (e.g., padding tokens).
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replaces any -100 values in labels with the tokenizer's pad_token_id.
    # This is done because -100 is often used to ignore certain tokens when calculating the loss during training.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decodes the tokenized labels back to text, skipping any special tokens (e.g., padding tokens).
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Computes the ROUGE metric between the decoded predictions and decoded labels.
    # The use_stemmer parameter enables stemming, which reduces words to their root form before comparison.
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Calculates the length of each prediction by counting the non-padding tokens.
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    # Computes the mean length of the predictions and adds it to the result dictionary under the key "gen_len".
    result["gen_len"] = np.mean(prediction_lens)

    # Rounds each value in the result dictionary to 4 decimal places for cleaner output, and returns the result.
    return {k: round(v, 4) for k, v in result.items()}

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")
training_args = Seq2SeqTrainingArguments(
    output_dir="my_fine_tuned_bart_base_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model("my_fine_tuned_bart_base_model")

Map: 100%|██████████| 200/200 [00:00<00:00, 11364.83 examples/s]
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\transformers\models\bart\modeling_bart.py:603: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\transformers\generation\utils.py:1252: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(













                                          
                                         

{'eval_loss': 1.148809552192688, 'eval_rouge1': 0.6801, 'eval_rouge2': 0.5133, 'eval_rougeL': 0.6739, 'eval_rougeLsum': 0.6735, 'eval_gen_len': 19.515, 'eval_runtime': 14.8866, 'eval_samples_per_second': 13.435, 'eval_steps_per_second': 0.873, 'epoch': 1.0}















                                          
                                              

  0%|          | 0/123798 [09:35<?, ?it/s]     



{'eval_loss': 0.9119622707366943, 'eval_rouge1': 0.7093, 'eval_rouge2': 0.5678, 'eval_rougeL': 0.7028, 'eval_rougeLsum': 0.7024, 'eval_gen_len': 19.58, 'eval_runtime': 14.6203, 'eval_samples_per_second': 13.68, 'eval_steps_per_second': 0.889, 'epoch': 2.0}















                                          
                                              

  0%|          | 0/123798 [09:53<?, ?it/s]     




{'eval_loss': 0.8096442818641663, 'eval_rouge1': 0.6929, 'eval_rouge2': 0.5645, 'eval_rougeL': 0.6868, 'eval_rougeLsum': 0.6863, 'eval_gen_len': 19.695, 'eval_runtime': 14.5695, 'eval_samples_per_second': 13.727, 'eval_steps_per_second': 0.892, 'epoch': 3.0}


 76%|███████▌  | 152/200 [00:55<01:42,  2.13s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
c:\Users\wjdrb\vscode_code\venv\Lib\site-packages\transformers\generation\utils.py:1252: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(













                                          
                                              

  0%|          | 0/123798 [10:13<?, ?it/s]     

                                          
100%|██████████| 20

{'eval_loss': 0.7830426096916199, 'eval_rouge1': 0.6919, 'eval_rouge2': 0.5649, 'eval_rougeL': 0.6873, 'eval_rougeLsum': 0.6873, 'eval_gen_len': 19.69, 'eval_runtime': 14.9184, 'eval_samples_per_second': 13.406, 'eval_steps_per_second': 0.871, 'epoch': 4.0}
{'train_runtime': 75.2482, 'train_samples_per_second': 42.526, 'train_steps_per_second': 2.658, 'train_loss': 1.272505645751953, 'epoch': 4.0}
